<a href="https://colab.research.google.com/github/josenalde/datascience/blob/main/socialnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lista de usuários definida como um dicionário (estrutura) com id e nome

In [ ]:
users = [
         {"id": 0, "name": "Jorge"},
         {"id": 1, "name": "Paulo"},
         {"id": 2, "name": "Lucas"},
         {"id": 3, "name": "Carlos"},
         {"id": 4, "name": "Pedro"},
         {"id": 5, "name": "Timoteo"},
         {"id": 6, "name": "Elias"},
         {"id": 7, "name": "Eliseu"},
         {"id": 8, "name": "Davi"},
         {"id": 9, "name": "Saul"}
]

Grafo de conexões diretas entre os usuários

In [ ]:
friendships = [(0,1), (0,2), (1,2), (1,3), (2,3), (3,4),
               (4,5),(5,6),(5,7),(6,8),(7,8), (8,9)]

Por exemplo, o user id0 Jorge é amigo de Lucas e de Paulo. Como a amizade é bidirecional, podemos criar uma lista de amigos para cada usuário.


In [ ]:
for user in users:
  user["friends"] = []
for i, j in friendships:
  users[i]["friends"].append(users[j]) # adiciona i como um amigo de j
  users[j]["friends"].append(users[i]) # adiciona j como um amigo de i
  print(i, j)

0 1
0 2
1 2
1 3
2 3
3 4
4 5
5 6
5 7
6 8
7 8
8 9


Agora, vamos determinar o número médio de conexões, com base no total de conexões dividido pelo número de usuários:


In [ ]:
#funcao para determinar o numero de amigos de um usuario
def number_of_friends(user):
  return len(user["friends"])

total_connections = sum(number_of_friends(user)
                        for user in users)
#from __future__ import division
num_users = len(users)
avg_connections = total_connections / num_users
print(avg_connections)

2.4


Encontrar o número de conexões de cada usuário:

In [ ]:
#cria uma lista (user_id, number_of_friends)
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]
print(num_friends_by_id)

[(0, 2), (1, 3), (2, 3), (3, 3), (4, 2), (5, 3), (6, 2), (7, 2), (8, 3), (9, 1)]


E ordenar do "mais amigos" ao "menos amigos". O uso do sorted() permite não alterar a lista original; senão usar .sort(), mas funciona apenas para listas. sorted() recebe qualquer objeto iterável, e sempre retorna uma lista. [Sobre sorted](https://towardsdatascience.com/sorting-lists-in-python-31477e0817d8)

In [ ]:
ordered_by_id=sorted(num_friends_by_id, reverse=True) #ordena por id de usuário
print(ordered_by_id)
ordered_by_n_friends = sorted(num_friends_by_id, key=lambda user_friends: user_friends[1], reverse=True)
print(ordered_by_n_friends)

[(9, 1), (8, 3), (7, 2), (6, 2), (5, 3), (4, 2), (3, 3), (2, 3), (1, 3), (0, 2)]
[(1, 3), (2, 3), (3, 3), (5, 3), (8, 3), (0, 2), (4, 2), (6, 2), (7, 2), (9, 1)]


Para aprofundar as consultas, pode-se encontrar amigos de amigos. Uma primeira solução descrita abaixo, gera duplicações, pois, por exemplo, 0 é amigo de 2, mas 2 é amigo de 0 e de 3, portanto duplica 0.

In [ ]:
def friends_of_friend_ids_bad(user):
  # foaf significa friend of a friend
  return [foaf["id"]
          for friend in user["friends"]
          for foaf in friend["friends"]]
print(friends_of_friend_ids_bad(users[0]))

[0, 2, 3, 0, 1, 3]


Vamos fazer então uma contagem de amigos em comum, que exclua pessoas que já são conhecidas do usuário

In [ ]:
from collections import Counter # para realizar contagem
# testa se os usuários são diferentes
def not_the_same(user, other_user):
  #dois usuários não são os mesmos se possuem ids diferentes
  return user["id"] != other_user["id"]

def not_friends(user, other_user):
  #other_user não é um amigo se não está em user["friends"], ou seja, se
  # é not_the_same com todas as pessoas em user["friends"]
  return all(not_the_same(friend, other_user)
             for friend in user["friends"])

# para cada um dos meus amigos que contam os amigos deles, que não sejam eu e que não sejam meus amigos
def friends_of_friends_ids(user):
  return Counter(foaf["id"]
                 for friend in user["friends"]
                 for foaf in friend["friends"]
                 if not_the_same(user, foaf)
                 and not_friends(user, foaf))
print(friends_of_friends_ids(users[3]))


Counter({0: 2, 5: 1})


Ou seja, o usuário 3 possui dois amigos em comum com 0 usuário 0 (2,1) e 1 amigo em comum com o usuário 5 (4)